In [59]:
from app.main.models import EducationStaff
import requests
from config import ApeksAPI
from app.main.func import db_filter_req, db_request
from datetime import date
from calendar import monthrange


def get_lessons(year, month):
    """List of lessons for selected month"""
    first_day = 1
    last_day = monthrange(year, month)[1]

    params = {
        "token": ApeksAPI.TOKEN,
        "table": 'schedule_day_schedule_lessons',
        "filter": "date between '" + date(year, month, first_day).isoformat() + "' and '" + date(year, month, last_day).isoformat() + "'"
    }
    resp = requests.get(
        ApeksAPI.URL + "/api/call/system-database/get", params=params)
    return resp.json()['data']


def get_departments():
    """Getting list of department as dict id:[name, short_name]"""
    dept_list = {}
    resp = db_filter_req('state_departments', 'parent_id', ApeksAPI.APEKS_DEPT_ID)
    for dept in resp:
        dept_list[dept['id']] = [dept['name'], dept['name_short']]
    return dept_list


def get_group_id(subgroup_id):
    """Get group_id from subgroup_id"""
    resp = db_filter_req('load_subgroups', 'id', subgroup_id)
    if resp:
        return resp[0].get('group_id')
    else:
        return None


def get_education_plan_id(group_id):
    """Get education_plan_id from group_id"""
    resp = db_filter_req('load_groups', 'id', group_id)
    if resp:
        return resp[0].get('education_plan_id')
    else:
        return None


def get_education_form_id(education_plan_id):
    resp = db_filter_req('plan_education_plans_education_forms', 'education_plan_id', education_plan_id)
    if resp:
        return resp[0].get('education_form_id')
    else:
        return None


def get_education_level_id(education_plan_id):
    resp = db_filter_req('plan_education_plans', 'id', education_plan_id)
    if resp:
        return resp[0].get('education_level_id')
    else:
        return None


class LoadReport:
    def __init__(self, year, month):
        self.year = year
        self.month = month
        self.departments = get_departments()
        self.structure = self.structure()
        # self.lessons = get_lessons(self.year, self.month)

    def structure(self):
        structure = {}
        for d in self.departments:
            staff = EducationStaff(self.year, self.month, d)
            staff_list = staff.staff_list()
            for s in staff_list:
                structure[s] = d
            # structure[d] = staff_list.staff_list()
        return structure


In [60]:
load = LoadReport(2021, 12)

In [78]:
staff_list = EducationStaff(2012, 12, 9)

In [79]:
staff_list.staff_list()

In [3]:
load.structure

{'6': '9',
 '7': '9',
 '10': '9',
 '11': '9',
 '9': '9',
 '12': '9',
 '13': '9',
 '8': '9',
 '155': '9',
 '160': '9',
 '289': '9',
 '24': '10',
 '27': '10',
 '28': '10',
 '29': '10',
 '25': '10',
 '26': '10',
 '53': '10',
 '16': '11',
 '119': '11',
 '120': '11',
 '17': '11',
 '18': '11',
 '21': '11',
 '19': '11',
 '20': '11',
 '22': '11',
 '23': '11',
 '30': '12',
 '32': '12',
 '33': '12',
 '54': '12',
 '34': '13',
 '35': '13',
 '36': '13',
 '38': '13',
 '161': '13',
 '37': '13',
 '39': '13',
 '40': '13',
 '41': '13',
 '42': '13',
 '156': '13',
 '162': '13',
 '163': '13',
 '286': '13',
 '55': '14',
 '56': '14',
 '57': '14',
 '58': '14',
 '59': '14',
 '60': '14',
 '61': '14',
 '62': '14',
 '67': '15',
 '292': '15',
 '65': '15',
 '68': '15',
 '69': '15',
 '70': '15',
 '71': '15',
 '72': '15',
 '73': '15',
 '74': '15',
 '133': '15',
 '167': '15',
 '80': '16',
 '81': '16',
 '82': '16',
 '83': '16',
 '121': '16',
 '43': '17',
 '44': '17',
 '45': '17',
 '46': '17',
 '47': '17',
 '48': '17',


In [62]:
lessons = get_lessons(2021, 12)

In [35]:
get_group_id(19)

'15'

In [63]:
structured_lessons = []
unknown_lessons = []
counter = 0
for lesson in lessons:
    counter += 1
    print(counter)
    lesson_staff = db_filter_req('schedule_day_schedule_lessons_staff', 'lesson_id', lesson.get('id'))
    if not lesson.get('group_id'):
        if lesson.get('subgroup_id'):
            lesson['group_id'] = get_group_id(lesson['subgroup_id'])
    education_plan_id = get_education_plan_id(lesson['group_id'])
    education_form_id = get_education_form_id(education_plan_id)
    education_level_id = get_education_level_id(education_plan_id)
    lesson['education_plan_id'] = education_plan_id
    lesson['education_form_id'] = education_form_id
    lesson['education_level_id'] = education_level_id
    for i in lesson_staff:
        if load.structure.get(i['staff_id']):
            lesson['staff_id'] = i['staff_id']
            lesson['department_id'] = load.structure.get(i['staff_id'])
            structured_lessons.append(lesson)
        else:
            lesson['staff_id'] = i['staff_id']
            unknown_lessons.append(lesson)


1
2
3
4
5
6
7
11
7
8
12
7
9
10
11
14
8
12
13
8
13
14
15
17
12
16
18
12
17
18
19
20
21
22
23
24
25
7
4
26
8
4
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
45
30
82
46
30
83
45
30
84
46
30
85
45
30
86
46
30
87
88
89
90
91
49
32
92
50
32
93
49
32
94
50
32
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
124
84
129
124
84
130
124
84
131
125
84
132
125
84
133
125
84
134
147
84
135
147
84
136
147
84
137
146
84
138
146
84
139
146
84
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
20
15
165
19
15
166
167
168
169
170
14
8
171
13
8
172
173
174
175
176
177
178
179
180
181
6
3
182
5
3
183
184
185
186
187
188
10
5
189
9
5
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
2

KeyboardInterrupt: 

In [58]:
unknown_lessons

[{'id': '9190',
  'schedule_id': '87',
  'discipline_id': '15',
  'class_type_id': '3',
  'control_type_id': None,
  'date': '2021-12-10',
  'lesson_time_id': '1',
  'topic_code': '7',
  'topic_name': 'Обработка табличных электронных документов',
  'classroom_id': '1',
  'group_id': '15',
  'subgroup_id': '19',
  'flow': None,
  'is_empty': '0',
  'self_work': '0',
  'skip_classroom_check': '0',
  'fixed': '1',
  'journal_lesson_id': '6891',
  'education_plan_id': '49',
  'education_form_id': '1',
  'education_level_id': '5',
  'staff_id': '31'},
 {'id': '9191',
  'schedule_id': '87',
  'discipline_id': '15',
  'class_type_id': '3',
  'control_type_id': None,
  'date': '2021-12-10',
  'lesson_time_id': '2',
  'topic_code': '7',
  'topic_name': 'Обработка табличных электронных документов',
  'classroom_id': '1',
  'group_id': '15',
  'subgroup_id': '19',
  'flow': None,
  'is_empty': '0',
  'self_work': '0',
  'skip_classroom_check': '0',
  'fixed': '1',
  'journal_lesson_id': '17988',


In [25]:
get_education_level_id(50)

KeyError: 'data'

In [16]:
get_education_plan_id(15)

'49'

In [97]:
structured_lessons

[{'id': '9037',
  'schedule_id': '89',
  'discipline_id': '4',
  'class_type_id': '3',
  'control_type_id': None,
  'date': '2021-12-01',
  'lesson_time_id': '1',
  'topic_code': '10.',
  'topic_name': 'Город, в котором получаешь высшее образование.',
  'classroom_id': '102',
  'group_id': '17',
  'subgroup_id': None,
  'flow': None,
  'is_empty': '0',
  'self_work': '0',
  'skip_classroom_check': '0',
  'fixed': '1',
  'journal_lesson_id': '7483',
  'staff_id': '287',
  'department_id': '22'},
 {'id': '9049',
  'schedule_id': '89',
  'discipline_id': '36',
  'class_type_id': '1',
  'control_type_id': None,
  'date': '2021-12-01',
  'lesson_time_id': '2',
  'topic_code': '17',
  'topic_name': 'Деятельность органов внутренних дел в борьбе с преступностью',
  'classroom_id': '102',
  'group_id': '17',
  'subgroup_id': None,
  'flow': None,
  'is_empty': '0',
  'self_work': '0',
  'skip_classroom_check': '0',
  'fixed': '1',
  'journal_lesson_id': '7484',
  'staff_id': '19',
  'department

In [100]:
db_filter_req('schedule_day_schedule_lessons_staff', 'lesson_id', 9628)

[{'lesson_id': '9628', 'staff_id': '33'}]

In [ ]:
124